# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Prompt for GPT
prompt_raw = "I would like to summarise in maximum 300 characters and in English what the following sentences in Portuguese talk about. \
Do not use aspects mentioned only in one sentence as part of the summary: TWEETS"

# 2014 negative

## January

In [4]:
# Get monthly data
df_2014_01_neg = pd.read_csv('../data/local/df_neg_2014-01.csv').reset_index()
df_2014_01_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   820 non-null    int64  
 1   LocationCode            820 non-null    object 
 2   LocationName            820 non-null    object 
 3   StatesCode              820 non-null    object 
 4   ...4                    820 non-null    int64  
 5   id                      820 non-null    float64
 6   text                    820 non-null    object 
 7   label                   820 non-null    object 
 8   in_reply_to_status_id   89 non-null     float64
 9   in_reply_to_user_id     97 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       120 non-null    float64
 13  retweeted_status_id     120 non-null    float64
 14  created_at              820 non-null    ob

### Representative sample based on week and location

In [5]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_01_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_01_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w01,Alagoas,1,0.001220
1,2014-w01,Amapa,1,0.001220
2,2014-w01,Amazonas,5,0.006098
3,2014-w01,Bahia,9,0.010976
4,2014-w01,Ceara,2,0.002439
...,...,...,...,...
82,2014-w05,Rio Grande Do Sul,16,0.019512
83,2014-w05,Rondonia,1,0.001220
84,2014-w05,Santa Catarina,5,0.006098
85,2014-w05,Sao Paulo,59,0.071951


In [6]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w01,Alagoas,1,0.001220,2
1,2014-w01,Amapa,1,0.001220,2
2,2014-w01,Amazonas,5,0.006098,14
3,2014-w01,Bahia,9,0.010976,26
4,2014-w01,Ceara,2,0.002439,5
...,...,...,...,...,...
82,2014-w05,Rio Grande Do Sul,16,0.019512,47
83,2014-w05,Rondonia,1,0.001220,2
84,2014-w05,Santa Catarina,5,0.006098,14
85,2014-w05,Sao Paulo,59,0.071951,176


In [7]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_01_neg_sample_size = pd.merge(df_2014_01_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [8]:
# Step 6: Perform the stratified sampling
#df_2014_01_neg_sample = df_2014_01_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_01_neg_sample = df_2014_01_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_01_neg_sample)} rows.")


Sampled 820 rows.


In [9]:
text_gpt_2014_01_neg = '\n'.join(df_2014_01_neg_sample['text'].astype(str))

### Prompts and loops

In [10]:
prompt_1 = prompt_raw.replace('TWEETS', text_gpt_2014_01_neg)

In [11]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_1}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [12]:
response.choices[0].message.content

"The sentences express various opinions and experiences regarding vaccinations, particularly the HPV vaccine and other childhood immunizations. Many individuals express fear, pain, and reluctance associated with getting vaccinated, while some criticize the government's vaccination campaigns as politically motivated. There are also mentions of adverse reactions to vaccines and concerns about their necessity and safety, particularly regarding HPV vaccinations for young girls. The overall sentiment reflects a mix of skepticism, fear, and frustration surrounding the topic of vaccinations."

In [13]:
summary = []
dataset = []

In [14]:
dataset.append("2014_01_negative")

In [15]:
summary.append(response.choices[0].message.content)

In [16]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1 non-null      object
 1   summary  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [17]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...


In [18]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdFfiW8t9esqMqzRWprbKzPHfsQ3 at 0x26c2b6a87c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express various opinions and experiences regarding vaccinations, particularly the HPV vaccine and other childhood immunizations. Many individuals express fear, pain, and reluctance associated with getting vaccinated, while some criticize the government's vaccination campaigns as politically motivated. There are also mentions of adverse reactions to vaccines and concerns about their necessity and safety, particularly regarding HPV vaccinations for young girls. The overall sentiment reflects a mix of skepticism, fear, and frustration surrounding the topic of vaccinations.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864087,
  "id": "chatcmpl-AUdFfiW8t9esqMqzRWprbKzPHfsQ3",
  "model": "gpt-4o-mini-2024-

## February

In [19]:
# Get monthly data
df_2014_02_neg = pd.read_csv('../data/local/df_neg_2014-02.csv').reset_index()
df_2014_02_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   738 non-null    int64  
 1   LocationCode            738 non-null    object 
 2   LocationName            738 non-null    object 
 3   StatesCode              738 non-null    object 
 4   ...4                    738 non-null    int64  
 5   id                      738 non-null    float64
 6   text                    738 non-null    object 
 7   label                   738 non-null    object 
 8   in_reply_to_status_id   70 non-null     float64
 9   in_reply_to_user_id     72 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       114 non-null    float64
 13  retweeted_status_id     114 non-null    float64
 14  created_at              738 non-null    ob

### Representative sample based on week and location

In [20]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_02_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_02_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w05,Acre,1,0.001355
1,2014-w05,Alagoas,1,0.001355
2,2014-w05,Amazonas,3,0.004065
3,2014-w05,Bahia,5,0.006775
4,2014-w05,Ceara,8,0.010840
...,...,...,...,...
85,2014-w09,Paraiba,3,0.004065
86,2014-w09,Parana,3,0.004065
87,2014-w09,Rio De Janeiro,1,0.001355
88,2014-w09,Rio Grande Do Sul,9,0.012195


In [21]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w05,Acre,1,0.001355,3
1,2014-w05,Alagoas,1,0.001355,3
2,2014-w05,Amazonas,3,0.004065,9
3,2014-w05,Bahia,5,0.006775,16
4,2014-w05,Ceara,8,0.010840,26
...,...,...,...,...,...
85,2014-w09,Paraiba,3,0.004065,9
86,2014-w09,Parana,3,0.004065,9
87,2014-w09,Rio De Janeiro,1,0.001355,3
88,2014-w09,Rio Grande Do Sul,9,0.012195,29


In [22]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_02_neg_sample_size = pd.merge(df_2014_02_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [23]:
# Step 6: Perform the stratified sampling
#df_2014_02_neg_sample = df_2014_02_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_02_neg_sample = df_2014_02_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_02_neg_sample)} rows.")


Sampled 738 rows.


In [24]:
text_gpt_2014_02_neg = '\n'.join(df_2014_02_neg_sample['text'].astype(str))

### Prompts and loops

In [25]:
prompt_2 = prompt_raw.replace('TWEETS', text_gpt_2014_02_neg)

In [26]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_2}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [27]:
response.choices[0].message.content

'The text discusses various opinions and experiences related to vaccinations, particularly skepticism towards the effectiveness of certain vaccines, such as the Cuban vaccine for meningitis B, and the HPV vaccine. There are concerns about side effects, parental opposition, and the role of religion in vaccine hesitancy. Additionally, experiences of pain and discomfort from vaccinations are expressed, alongside criticisms of the anti-vaccine movement and its consequences on public health.'

In [28]:
#summary = []
#dataset = []

In [29]:
dataset.append("2014_02_negative")

In [30]:
summary.append(response.choices[0].message.content)

In [31]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  2 non-null      object
 1   summary  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [32]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...


In [33]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdFhPFhO7iinEvkpTkFM2cm8dMZn at 0x26c2b8bedb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences related to vaccinations, particularly skepticism towards the effectiveness of certain vaccines, such as the Cuban vaccine for meningitis B, and the HPV vaccine. There are concerns about side effects, parental opposition, and the role of religion in vaccine hesitancy. Additionally, experiences of pain and discomfort from vaccinations are expressed, alongside criticisms of the anti-vaccine movement and its consequences on public health.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864089,
  "id": "chatcmpl-AUdFhPFhO7iinEvkpTkFM2cm8dMZn",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "co

## March

In [34]:
# Get monthly data
df_2014_03_neg = pd.read_csv('../data/local/df_neg_2014-03.csv').reset_index()
df_2014_03_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3604 entries, 0 to 3603
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   3604 non-null   int64  
 1   LocationCode            3604 non-null   object 
 2   LocationName            3604 non-null   object 
 3   StatesCode              3604 non-null   object 
 4   ...4                    3604 non-null   int64  
 5   id                      3604 non-null   float64
 6   text                    3604 non-null   object 
 7   label                   3604 non-null   object 
 8   in_reply_to_status_id   292 non-null    float64
 9   in_reply_to_user_id     328 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       516 non-null    float64
 13  retweeted_status_id     516 non-null    float64
 14  created_at              3604 non-null   

### Representative sample based on week and location

In [35]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_03_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_03_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w09,Alagoas,1,0.000277
1,2014-w09,Amazonas,1,0.000277
2,2014-w09,Ceara,2,0.000555
3,2014-w09,Distrito Federal,1,0.000277
4,2014-w09,Espirito Santo,1,0.000277
...,...,...,...,...
106,2014-w13,Rio Grande Do Sul,140,0.038846
107,2014-w13,Roraima,2,0.000555
108,2014-w13,Santa Catarina,25,0.006937
109,2014-w13,Sao Paulo,106,0.029412


In [36]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w09,Alagoas,1,0.000277,0
1,2014-w09,Amazonas,1,0.000277,0
2,2014-w09,Ceara,2,0.000555,1
3,2014-w09,Distrito Federal,1,0.000277,0
4,2014-w09,Espirito Santo,1,0.000277,0
...,...,...,...,...,...
106,2014-w13,Rio Grande Do Sul,140,0.038846,95
107,2014-w13,Roraima,2,0.000555,1
108,2014-w13,Santa Catarina,25,0.006937,16
109,2014-w13,Sao Paulo,106,0.029412,72


In [37]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_03_neg_sample_size = pd.merge(df_2014_03_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [38]:
# Step 6: Perform the stratified sampling
#df_2014_03_neg_sample = df_2014_03_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_03_neg_sample = df_2014_03_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_03_neg_sample)} rows.")


Sampled 3604 rows.


In [39]:
text_gpt_2014_03_neg = '\n'.join(df_2014_03_neg_sample['text'].astype(str))

### Prompts and loops

In [40]:
prompt_3 = prompt_raw.replace('TWEETS', text_gpt_2014_03_neg)

In [41]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_3}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [42]:
response.choices[0].message.content

'The text discusses the controversy surrounding the HPV vaccination, particularly focusing on evangelical mothers who are reportedly boycotting the vaccine for their daughters, believing it promotes early sexual activity. Many individuals express fears about the pain associated with the vaccine and its potential side effects, with reports of adverse reactions in some cases. The text highlights the tension between public health initiatives and personal beliefs, particularly in religious communities.'

In [43]:
#summary = []
#dataset = []

In [44]:
dataset.append("2014_03_negative")

In [45]:
summary.append(response.choices[0].message.content)

In [46]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  3 non-null      object
 1   summary  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [47]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...


In [48]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdFlp9inoSd5T522W32RTx5Y4eRr at 0x26c2ba92ae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses the controversy surrounding the HPV vaccination, particularly focusing on evangelical mothers who are reportedly boycotting the vaccine for their daughters, believing it promotes early sexual activity. Many individuals express fears about the pain associated with the vaccine and its potential side effects, with reports of adverse reactions in some cases. The text highlights the tension between public health initiatives and personal beliefs, particularly in religious communities.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864093,
  "id": "chatcmpl-AUdFlp9inoSd5T522W32RTx5Y4eRr",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage

## April

In [49]:
# Get monthly data
df_2014_04_neg = pd.read_csv('../data/local/df_neg_2014-04.csv').reset_index()
df_2014_04_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1545 entries, 0 to 1544
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1545 non-null   int64  
 1   LocationCode            1545 non-null   object 
 2   LocationName            1545 non-null   object 
 3   StatesCode              1545 non-null   object 
 4   ...4                    1545 non-null   int64  
 5   id                      1545 non-null   float64
 6   text                    1545 non-null   object 
 7   label                   1545 non-null   object 
 8   in_reply_to_status_id   106 non-null    float64
 9   in_reply_to_user_id     118 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       137 non-null    float64
 13  retweeted_status_id     137 non-null    float64
 14  created_at              1545 non-null   

### Representative sample based on week and location

In [50]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_04_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_04_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w13,Acre,1,0.000647
1,2014-w13,Amapa,1,0.000647
2,2014-w13,Bahia,5,0.003236
3,2014-w13,Ceara,2,0.001294
4,2014-w13,Distrito Federal,1,0.000647
...,...,...,...,...
106,2014-w18,Pernambuco,2,0.001294
107,2014-w18,Rio Grande Do Sul,17,0.011003
108,2014-w18,Santa Catarina,4,0.002589
109,2014-w18,Sao Paulo,14,0.009061


In [51]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w13,Acre,1,0.000647,1
1,2014-w13,Amapa,1,0.000647,1
2,2014-w13,Bahia,5,0.003236,7
3,2014-w13,Ceara,2,0.001294,3
4,2014-w13,Distrito Federal,1,0.000647,1
...,...,...,...,...,...
106,2014-w18,Pernambuco,2,0.001294,3
107,2014-w18,Rio Grande Do Sul,17,0.011003,26
108,2014-w18,Santa Catarina,4,0.002589,6
109,2014-w18,Sao Paulo,14,0.009061,22


In [52]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_04_neg_sample_size = pd.merge(df_2014_04_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [53]:
# Step 6: Perform the stratified sampling
#df_2014_04_neg_sample = df_2014_04_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_04_neg_sample = df_2014_04_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_04_neg_sample)} rows.")


Sampled 1545 rows.


In [54]:
text_gpt_2014_04_neg = '\n'.join(df_2014_04_neg_sample['text'].astype(str))

### Prompts and loops

In [55]:
prompt_4 = prompt_raw.replace('TWEETS', text_gpt_2014_04_neg)

In [56]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_4}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [57]:
response.choices[0].message.content

"The sentences discuss various sentiments about vaccinations, particularly focusing on the pain and discomfort experienced after receiving them, the fear and avoidance some people feel towards vaccinations, and the ongoing campaigns for vaccines like HPV and flu shots. Many express annoyance at having to take these vaccines, while others mention complications or adverse reactions following vaccination. There's also mention of issues like vaccine shortages and controversies surrounding the necessity of certain vaccines. Overall, there's a mix of frustration, pain, and societal discussion regarding vaccinations."

In [58]:
#summary = []
#dataset = []

In [59]:
dataset.append("2014_04_negative")

In [60]:
summary.append(response.choices[0].message.content)

In [61]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  4 non-null      object
 1   summary  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [62]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...


In [63]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdFvM3ntagoJSbN1p85s4yX68FOA at 0x26c2bbf4810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss various sentiments about vaccinations, particularly focusing on the pain and discomfort experienced after receiving them, the fear and avoidance some people feel towards vaccinations, and the ongoing campaigns for vaccines like HPV and flu shots. Many express annoyance at having to take these vaccines, while others mention complications or adverse reactions following vaccination. There's also mention of issues like vaccine shortages and controversies surrounding the necessity of certain vaccines. Overall, there's a mix of frustration, pain, and societal discussion regarding vaccinations.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864103,
  "id": "chatcmpl-AUdFvM3ntagoJSbN1p85s4yX68FOA",
  "

## May

In [64]:
# Get monthly data
df_2014_05_neg = pd.read_csv('../data/local/df_neg_2014-05.csv').reset_index()
df_2014_05_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2017 non-null   int64  
 1   LocationCode            2017 non-null   object 
 2   LocationName            2017 non-null   object 
 3   StatesCode              2017 non-null   object 
 4   ...4                    2017 non-null   int64  
 5   id                      2017 non-null   float64
 6   text                    2017 non-null   object 
 7   label                   2017 non-null   object 
 8   in_reply_to_status_id   122 non-null    float64
 9   in_reply_to_user_id     137 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       179 non-null    float64
 13  retweeted_status_id     179 non-null    float64
 14  created_at              2017 non-null   

### Representative sample based on week and location

In [65]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_05_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_05_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w18,Acre,1,0.000496
1,2014-w18,Alagoas,6,0.002975
2,2014-w18,Amazonas,6,0.002975
3,2014-w18,Bahia,20,0.009916
4,2014-w18,Ceara,10,0.004958
...,...,...,...,...
98,2014-w22,Rio De Janeiro,1,0.000496
99,2014-w22,Rio Grande Do Sul,34,0.016857
100,2014-w22,Santa Catarina,9,0.004462
101,2014-w22,Sao Paulo,26,0.012890


In [66]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w18,Acre,1,0.000496,1
1,2014-w18,Alagoas,6,0.002975,7
2,2014-w18,Amazonas,6,0.002975,7
3,2014-w18,Bahia,20,0.009916,24
4,2014-w18,Ceara,10,0.004958,12
...,...,...,...,...,...
98,2014-w22,Rio De Janeiro,1,0.000496,1
99,2014-w22,Rio Grande Do Sul,34,0.016857,41
100,2014-w22,Santa Catarina,9,0.004462,10
101,2014-w22,Sao Paulo,26,0.012890,31


In [67]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_05_neg_sample_size = pd.merge(df_2014_05_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [68]:
# Step 6: Perform the stratified sampling
#df_2014_05_neg_sample = df_2014_05_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_05_neg_sample = df_2014_05_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_05_neg_sample)} rows.")


Sampled 2017 rows.


In [69]:
text_gpt_2014_05_neg = '\n'.join(df_2014_05_neg_sample['text'].astype(str))

### Prompts and loops

In [70]:
prompt_5 = prompt_raw.replace('TWEETS', text_gpt_2014_05_neg)

In [71]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_5}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [72]:
response.choices[0].message.content

'The text discusses various opinions and experiences related to vaccinations, particularly the flu vaccine. Many individuals express their dislike for vaccines, recounting painful experiences and side effects, such as soreness in the arm, feeling ill, or even getting the flu after vaccination. Some mention the fear of needles and the pressure to get vaccinated, while others criticize the effectiveness of vaccines. There are also references to the lack of available vaccines in certain areas and concerns regarding the safety and reliability of vaccinations, including controversies surrounding specific vaccines like HPV. Overall, the sentiment is mixed, with many expressing skepticism and discomfort about the vaccination process.'

In [73]:
#summary = []
#dataset = []

In [74]:
dataset.append("2014_05_negative")

In [75]:
summary.append(response.choices[0].message.content)

In [76]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  5 non-null      object
 1   summary  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [77]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...


In [78]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdG0NoRKlcBBCw8qcNmjqTHKoBFP at 0x26c2bd8a900> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences related to vaccinations, particularly the flu vaccine. Many individuals express their dislike for vaccines, recounting painful experiences and side effects, such as soreness in the arm, feeling ill, or even getting the flu after vaccination. Some mention the fear of needles and the pressure to get vaccinated, while others criticize the effectiveness of vaccines. There are also references to the lack of available vaccines in certain areas and concerns regarding the safety and reliability of vaccinations, including controversies surrounding specific vaccines like HPV. Overall, the sentiment is mixed, with many expressing skepticism and discomfort about the vaccination process.",
        "refusal": null,
      

## June

In [79]:
# Get monthly data
df_2014_06_neg = pd.read_csv('../data/local/df_neg_2014-06.csv').reset_index()
df_2014_06_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   868 non-null    int64  
 1   LocationCode            868 non-null    object 
 2   LocationName            868 non-null    object 
 3   StatesCode              868 non-null    object 
 4   ...4                    868 non-null    int64  
 5   id                      868 non-null    float64
 6   text                    868 non-null    object 
 7   label                   868 non-null    object 
 8   in_reply_to_status_id   72 non-null     float64
 9   in_reply_to_user_id     77 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       141 non-null    float64
 13  retweeted_status_id     141 non-null    float64
 14  created_at              868 non-null    ob

### Representative sample based on week and location

In [80]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_06_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_06_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w22,Amazonas,2,0.002304
1,2014-w22,Bahia,1,0.001152
2,2014-w22,Ceara,1,0.001152
3,2014-w22,Distrito Federal,4,0.004608
4,2014-w22,Espirito Santo,2,0.002304
...,...,...,...,...
86,2014-w26,Parana,10,0.011521
87,2014-w26,Pernambuco,3,0.003456
88,2014-w26,Rio Grande Do Sul,23,0.026498
89,2014-w26,Santa Catarina,7,0.008065


In [81]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w22,Amazonas,2,0.002304,5
1,2014-w22,Bahia,1,0.001152,2
2,2014-w22,Ceara,1,0.001152,2
3,2014-w22,Distrito Federal,4,0.004608,11
4,2014-w22,Espirito Santo,2,0.002304,5
...,...,...,...,...,...
86,2014-w26,Parana,10,0.011521,28
87,2014-w26,Pernambuco,3,0.003456,8
88,2014-w26,Rio Grande Do Sul,23,0.026498,64
89,2014-w26,Santa Catarina,7,0.008065,19


In [82]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_06_neg_sample_size = pd.merge(df_2014_06_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [83]:
# Step 6: Perform the stratified sampling
#df_2014_06_neg_sample = df_2014_06_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_06_neg_sample = df_2014_06_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_06_neg_sample)} rows.")


Sampled 868 rows.


In [84]:
text_gpt_2014_06_neg = '\n'.join(df_2014_06_neg_sample['text'].astype(str))

### Prompts and loops

In [85]:
prompt_6 = prompt_raw.replace('TWEETS', text_gpt_2014_06_neg)

In [86]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_6}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [87]:
response.choices[0].message.content

'The text discusses various public reactions and complaints regarding vaccination campaigns in Brazil, particularly against the flu and HPV. Many individuals express frustration about side effects, ineffectiveness of the vaccines, and the lack of availability. There are also mentions of fears surrounding vaccinations and a general skepticism towards their benefits, reflecting concerns about public health policies and government accountability.'

In [88]:
#summary = []
#dataset = []

In [89]:
dataset.append("2014_06_negative")

In [90]:
summary.append(response.choices[0].message.content)

In [91]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  6 non-null      object
 1   summary  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


In [92]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...


In [93]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdG6ha8nUZUO8dtXA6Sgm4tHvDJT at 0x26c2bf83f40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various public reactions and complaints regarding vaccination campaigns in Brazil, particularly against the flu and HPV. Many individuals express frustration about side effects, ineffectiveness of the vaccines, and the lack of availability. There are also mentions of fears surrounding vaccinations and a general skepticism towards their benefits, reflecting concerns about public health policies and government accountability.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864114,
  "id": "chatcmpl-AUdG6ha8nUZUO8dtXA6Sgm4tHvDJT",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 68,
    "completion_tokens

## July

In [94]:
# Get monthly data
df_2014_07_neg = pd.read_csv('../data/local/df_neg_2014-07.csv').reset_index()
df_2014_07_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   639 non-null    int64  
 1   LocationCode            639 non-null    object 
 2   LocationName            639 non-null    object 
 3   StatesCode              639 non-null    object 
 4   ...4                    639 non-null    int64  
 5   id                      639 non-null    float64
 6   text                    639 non-null    object 
 7   label                   639 non-null    object 
 8   in_reply_to_status_id   57 non-null     float64
 9   in_reply_to_user_id     61 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       54 non-null     float64
 13  retweeted_status_id     54 non-null     float64
 14  created_at              639 non-null    ob

### Representative sample based on week and location

In [95]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_07_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_07_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w26,Bahia,1,0.001565
1,2014-w26,Espirito Santo,3,0.004695
2,2014-w26,Minas Gerais,7,0.010955
3,2014-w26,Paraiba,2,0.003130
4,2014-w26,Parana,4,0.006260
...,...,...,...,...
93,2014-w31,Parana,2,0.003130
94,2014-w31,Piaui,1,0.001565
95,2014-w31,Rio Grande Do Sul,12,0.018779
96,2014-w31,Santa Catarina,10,0.015649


In [96]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w26,Bahia,1,0.001565,3
1,2014-w26,Espirito Santo,3,0.004695,11
2,2014-w26,Minas Gerais,7,0.010955,26
3,2014-w26,Paraiba,2,0.003130,7
4,2014-w26,Parana,4,0.006260,15
...,...,...,...,...,...
93,2014-w31,Parana,2,0.003130,7
94,2014-w31,Piaui,1,0.001565,3
95,2014-w31,Rio Grande Do Sul,12,0.018779,46
96,2014-w31,Santa Catarina,10,0.015649,38


In [97]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_07_neg_sample_size = pd.merge(df_2014_07_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [98]:
# Step 6: Perform the stratified sampling
#df_2014_07_neg_sample = df_2014_07_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_07_neg_sample = df_2014_07_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_07_neg_sample)} rows.")


Sampled 639 rows.


In [99]:
text_gpt_2014_07_neg = '\n'.join(df_2014_07_neg_sample['text'].astype(str))

### Prompts and loops

In [100]:
prompt_7 = prompt_raw.replace('TWEETS', text_gpt_2014_07_neg)

In [101]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_7}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [102]:
response.choices[0].message.content

'The text discusses various experiences and opinions regarding vaccinations, particularly focusing on the flu vaccine and the HPV vaccine. Many individuals express discomfort, fear, or frustration about getting vaccinated, sharing reactions to side effects, and questioning the effectiveness of vaccines. There are also mentions of vaccination campaigns and public health issues related to vaccine availability.'

In [103]:
#summary = []
#dataset = []

In [104]:
dataset.append("2014_07_negative")

In [105]:
summary.append(response.choices[0].message.content)

In [106]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  7 non-null      object
 1   summary  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [107]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...
6,2014_07_negitive,The text discusses various experiences and opi...


In [108]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdG8AAGgRV6FYHkszEhAVXWnxsto at 0x26c2be84770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various experiences and opinions regarding vaccinations, particularly focusing on the flu vaccine and the HPV vaccine. Many individuals express discomfort, fear, or frustration about getting vaccinated, sharing reactions to side effects, and questioning the effectiveness of vaccines. There are also mentions of vaccination campaigns and public health issues related to vaccine availability.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864116,
  "id": "chatcmpl-AUdG8AAGgRV6FYHkszEhAVXWnxsto",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 63,
    "completion_tokens_details": {
      "accepted_predict

## August

In [109]:
# Get monthly data
df_2014_08_neg = pd.read_csv('../data/local/df_neg_2014-08.csv').reset_index()
df_2014_08_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   672 non-null    int64  
 1   LocationCode            672 non-null    object 
 2   LocationName            672 non-null    object 
 3   StatesCode              672 non-null    object 
 4   ...4                    672 non-null    int64  
 5   id                      672 non-null    float64
 6   text                    672 non-null    object 
 7   label                   672 non-null    object 
 8   in_reply_to_status_id   60 non-null     float64
 9   in_reply_to_user_id     70 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       109 non-null    float64
 13  retweeted_status_id     109 non-null    float64
 14  created_at              672 non-null    ob

### Representative sample based on week and location

In [110]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_08_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_08_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w31,Alagoas,1,0.001488
1,2014-w31,Amazonas,3,0.004464
2,2014-w31,Bahia,3,0.004464
3,2014-w31,Ceara,1,0.001488
4,2014-w31,Distrito Federal,3,0.004464
...,...,...,...,...
80,2014-w35,Parana,5,0.007440
81,2014-w35,Pernambuco,2,0.002976
82,2014-w35,Rio Grande Do Sul,17,0.025298
83,2014-w35,Santa Catarina,1,0.001488


In [111]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w31,Alagoas,1,0.001488,3
1,2014-w31,Amazonas,3,0.004464,10
2,2014-w31,Bahia,3,0.004464,10
3,2014-w31,Ceara,1,0.001488,3
4,2014-w31,Distrito Federal,3,0.004464,10
...,...,...,...,...,...
80,2014-w35,Parana,5,0.007440,18
81,2014-w35,Pernambuco,2,0.002976,7
82,2014-w35,Rio Grande Do Sul,17,0.025298,61
83,2014-w35,Santa Catarina,1,0.001488,3


In [112]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_08_neg_sample_size = pd.merge(df_2014_08_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [113]:
# Step 6: Perform the stratified sampling
#df_2014_08_neg_sample = df_2014_08_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_08_neg_sample = df_2014_08_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_08_neg_sample)} rows.")


Sampled 672 rows.


In [114]:
text_gpt_2014_08_neg = '\n'.join(df_2014_08_neg_sample['text'].astype(str))

### Prompts and loops

In [115]:
prompt_8 = prompt_raw.replace('TWEETS', text_gpt_2014_08_neg)

In [116]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_8}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [117]:
response.choices[0].message.content

'The text discusses various experiences and opinions about vaccinations, including personal pain and discomfort after receiving vaccines, criticism of vaccine availability in certain regions, and disparities in vaccine development for diseases affecting different populations. It highlights frustrations with mandatory vaccinations and the perceived ineffectiveness of some vaccines, especially in relation to diseases like Ebola and the flu.'

In [118]:
#summary = []
#dataset = []

In [119]:
dataset.append("2014_08_negative")

In [120]:
summary.append(response.choices[0].message.content)

In [121]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  8 non-null      object
 1   summary  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [122]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...
6,2014_07_negitive,The text discusses various experiences and opi...
7,2014_08_negitive,The text discusses various experiences and opi...


In [123]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdGBOcyLL01x8v0xWvkgLIUvWouE at 0x26c2d0e1f90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various experiences and opinions about vaccinations, including personal pain and discomfort after receiving vaccines, criticism of vaccine availability in certain regions, and disparities in vaccine development for diseases affecting different populations. It highlights frustrations with mandatory vaccinations and the perceived ineffectiveness of some vaccines, especially in relation to diseases like Ebola and the flu.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864119,
  "id": "chatcmpl-AUdGBOcyLL01x8v0xWvkgLIUvWouE",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 65,
    "completion_tokens_deta

## September

In [124]:
# Get monthly data
df_2014_09_neg = pd.read_csv('../data/local/df_neg_2014-09.csv').reset_index()
df_2014_09_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1917 non-null   int64  
 1   LocationCode            1917 non-null   object 
 2   LocationName            1917 non-null   object 
 3   StatesCode              1917 non-null   object 
 4   ...4                    1917 non-null   int64  
 5   id                      1917 non-null   float64
 6   text                    1917 non-null   object 
 7   label                   1917 non-null   object 
 8   in_reply_to_status_id   120 non-null    float64
 9   in_reply_to_user_id     129 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       389 non-null    float64
 13  retweeted_status_id     389 non-null    float64
 14  created_at              1917 non-null   

### Representative sample based on week and location

In [125]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_09_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_09_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w35,Alagoas,1,0.000522
1,2014-w35,Amazonas,2,0.001043
2,2014-w35,Bahia,4,0.002087
3,2014-w35,Ceara,5,0.002608
4,2014-w35,Espirito Santo,3,0.001565
...,...,...,...,...
100,2014-w39,Pernambuco,7,0.003652
101,2014-w39,Rio Grande Do Sul,50,0.026082
102,2014-w39,Santa Catarina,9,0.004695
103,2014-w39,Sao Paulo,46,0.023996


In [126]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w35,Alagoas,1,0.000522,1
1,2014-w35,Amazonas,2,0.001043,2
2,2014-w35,Bahia,4,0.002087,5
3,2014-w35,Ceara,5,0.002608,6
4,2014-w35,Espirito Santo,3,0.001565,3
...,...,...,...,...,...
100,2014-w39,Pernambuco,7,0.003652,8
101,2014-w39,Rio Grande Do Sul,50,0.026082,63
102,2014-w39,Santa Catarina,9,0.004695,11
103,2014-w39,Sao Paulo,46,0.023996,58


In [127]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_09_neg_sample_size = pd.merge(df_2014_09_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [128]:
# Step 6: Perform the stratified sampling
#df_2014_09_neg_sample = df_2014_09_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_09_neg_sample = df_2014_09_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_09_neg_sample)} rows.")


Sampled 1917 rows.


In [129]:
text_gpt_2014_09_neg = '\n'.join(df_2014_09_neg_sample['text'].astype(str))

### Prompts and loops

In [130]:
prompt_9 = prompt_raw.replace('TWEETS', text_gpt_2014_09_neg)

In [131]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_9}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [132]:
response.choices[0].message.content

"Several reports highlight serious adverse reactions to the HPV vaccine, with multiple young girls experiencing paralysis and hospitalization following vaccination. In particular, three adolescents in Bertioga have been reported to have lost movement in their legs, raising concerns about the vaccine's safety. Additionally, there are alarming reports of at least 15 child fatalities linked to a measles vaccination campaign in Syria, attributed to human error. The situation has sparked widespread fear and skepticism regarding the safety and efficacy of various vaccines."

In [133]:
#summary = []
#dataset = []

In [134]:
dataset.append("2014_09_negative")

In [135]:
summary.append(response.choices[0].message.content)

In [136]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  9 non-null      object
 1   summary  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [137]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...
6,2014_07_negitive,The text discusses various experiences and opi...
7,2014_08_negitive,The text discusses various experiences and opi...
8,2014_09_negitive,Several reports highlight serious adverse reac...


In [138]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdGT5yMfaES5doqFMLwAxKF6iwDq at 0x26c2d15c9f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Several reports highlight serious adverse reactions to the HPV vaccine, with multiple young girls experiencing paralysis and hospitalization following vaccination. In particular, three adolescents in Bertioga have been reported to have lost movement in their legs, raising concerns about the vaccine's safety. Additionally, there are alarming reports of at least 15 child fatalities linked to a measles vaccination campaign in Syria, attributed to human error. The situation has sparked widespread fear and skepticism regarding the safety and efficacy of various vaccines.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864137,
  "id": "chatcmpl-AUdGT5yMfaES5doqFMLwAxKF6iwDq",
  "model": "gpt-4o-mini-2024-07-18",
  "object"

## October

In [139]:
# Get monthly data
df_2014_10_neg = pd.read_csv('../data/local/df_neg_2014-10.csv').reset_index()
df_2014_10_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360 entries, 0 to 3359
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   3360 non-null   int64  
 1   LocationCode            3360 non-null   object 
 2   LocationName            3360 non-null   object 
 3   StatesCode              3360 non-null   object 
 4   ...4                    3360 non-null   int64  
 5   id                      3360 non-null   float64
 6   text                    3360 non-null   object 
 7   label                   3360 non-null   object 
 8   in_reply_to_status_id   251 non-null    float64
 9   in_reply_to_user_id     265 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       1768 non-null   float64
 13  retweeted_status_id     1768 non-null   float64
 14  created_at              3360 non-null   

### Representative sample based on week and location

In [140]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_10_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_10_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w40,Acre,1,0.000298
1,2014-w40,Alagoas,5,0.001488
2,2014-w40,Amapa,6,0.001786
3,2014-w40,Amazonas,3,0.000893
4,2014-w40,Bahia,16,0.004762
...,...,...,...,...
102,2014-w44,Parana,4,0.001190
103,2014-w44,Pernambuco,3,0.000893
104,2014-w44,Rio Grande Do Sul,8,0.002381
105,2014-w44,Sao Paulo,13,0.003869


In [141]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w40,Acre,1,0.000298,0
1,2014-w40,Alagoas,5,0.001488,3
2,2014-w40,Amapa,6,0.001786,4
3,2014-w40,Amazonas,3,0.000893,2
4,2014-w40,Bahia,16,0.004762,11
...,...,...,...,...,...
102,2014-w44,Parana,4,0.001190,2
103,2014-w44,Pernambuco,3,0.000893,2
104,2014-w44,Rio Grande Do Sul,8,0.002381,5
105,2014-w44,Sao Paulo,13,0.003869,9


In [142]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_10_neg_sample_size = pd.merge(df_2014_10_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [143]:
# Step 6: Perform the stratified sampling
#df_2014_10_neg_sample = df_2014_10_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_10_neg_sample = df_2014_10_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_10_neg_sample)} rows.")


Sampled 3360 rows.


In [144]:
text_gpt_2014_10_neg = '\n'.join(df_2014_10_neg_sample['text'].astype(str))

### Prompts and loops

In [145]:
prompt_10 = prompt_raw.replace('TWEETS', text_gpt_2014_10_neg)

In [146]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_10}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [147]:
response.choices[0].message.content

"The sentences discuss the controversy surrounding Aécio Neves' administration in Minas Gerais, particularly the alleged use of public health funds to purchase vaccines for horses, which some claim were misclassified as health expenditures. This has led to significant public outrage and ridicule, with many questioning the prioritization of animal health over human health. The debate has sparked comparisons to broader issues in Brazilian politics, highlighting concerns over corruption and mismanagement in the health sector."

In [148]:
#summary = []
#dataset = []

In [149]:
dataset.append("2014_10_negative")

In [150]:
summary.append(response.choices[0].message.content)

In [151]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  10 non-null     object
 1   summary  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [152]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...
6,2014_07_negitive,The text discusses various experiences and opi...
7,2014_08_negitive,The text discusses various experiences and opi...
8,2014_09_negitive,Several reports highlight serious adverse reac...
9,2014_10_negitive,The sentences discuss the controversy surround...


In [153]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdGZHfzAoYfmRRV2pHBlUxlCOVBq at 0x26c2d370360> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss the controversy surrounding A\u00e9cio Neves' administration in Minas Gerais, particularly the alleged use of public health funds to purchase vaccines for horses, which some claim were misclassified as health expenditures. This has led to significant public outrage and ridicule, with many questioning the prioritization of animal health over human health. The debate has sparked comparisons to broader issues in Brazilian politics, highlighting concerns over corruption and mismanagement in the health sector.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864143,
  "id": "chatcmpl-AUdGZHfzAoYfmRRV2pHBlUxlCOVBq",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerpri

## November

In [154]:
# Get monthly data
df_2014_11_neg = pd.read_csv('../data/local/df_neg_2014-11.csv').reset_index()
df_2014_11_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   863 non-null    int64  
 1   LocationCode            863 non-null    object 
 2   LocationName            863 non-null    object 
 3   StatesCode              863 non-null    object 
 4   ...4                    863 non-null    int64  
 5   id                      863 non-null    float64
 6   text                    863 non-null    object 
 7   label                   863 non-null    object 
 8   in_reply_to_status_id   63 non-null     float64
 9   in_reply_to_user_id     70 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       182 non-null    float64
 13  retweeted_status_id     182 non-null    float64
 14  created_at              863 non-null    ob

### Representative sample based on week and location

In [155]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_11_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_11_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w44,Acre,1,0.001159
1,2014-w44,Amazonas,2,0.002317
2,2014-w44,Bahia,6,0.006952
3,2014-w44,Ceara,9,0.010429
4,2014-w44,Distrito Federal,4,0.004635
...,...,...,...,...
92,2014-w48,Pernambuco,8,0.009270
93,2014-w48,Rio Grande Do Sul,9,0.010429
94,2014-w48,Santa Catarina,4,0.004635
95,2014-w48,Sao Paulo,22,0.025492


In [156]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w44,Acre,1,0.001159,2
1,2014-w44,Amazonas,2,0.002317,5
2,2014-w44,Bahia,6,0.006952,17
3,2014-w44,Ceara,9,0.010429,25
4,2014-w44,Distrito Federal,4,0.004635,11
...,...,...,...,...,...
92,2014-w48,Pernambuco,8,0.009270,22
93,2014-w48,Rio Grande Do Sul,9,0.010429,25
94,2014-w48,Santa Catarina,4,0.004635,11
95,2014-w48,Sao Paulo,22,0.025492,62


In [157]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_11_neg_sample_size = pd.merge(df_2014_11_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [158]:
# Step 6: Perform the stratified sampling
#df_2014_11_neg_sample = df_2014_11_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_11_neg_sample = df_2014_11_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_11_neg_sample)} rows.")


Sampled 863 rows.


In [159]:
text_gpt_2014_11_neg = '\n'.join(df_2014_11_neg_sample['text'].astype(str))

### Prompts and loops

In [160]:
prompt_11 = prompt_raw.replace('TWEETS', text_gpt_2014_11_neg)

In [161]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_11}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [162]:
response.choices[0].message.content

'The text discusses various aspects of vaccination, including the importance of keeping vaccinations up to date for children, the risks associated with certain vaccines for children with allergies (especially to milk), the low uptake of the HPV vaccine among girls, and reactions to the measles vaccine leading to its suspension in Amazonas. It also mentions criticism from the WHO regarding the lack of investment in vaccines for diseases like Ebola.'

In [163]:
#summary = []
#dataset = []

In [164]:
dataset.append("2014_11_negative")

In [165]:
summary.append(response.choices[0].message.content)

In [166]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  11 non-null     object
 1   summary  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [167]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negitive,The sentences express various opinions and exp...
1,2014_02_negitive,The text discusses various opinions and experi...
2,2014_03_negitive,The text discusses the controversy surrounding...
3,2014_04_negitive,The sentences discuss various sentiments about...
4,2014_05_negitive,The text discusses various opinions and experi...
5,2014_06_negitive,The text discusses various public reactions an...
6,2014_07_negitive,The text discusses various experiences and opi...
7,2014_08_negitive,The text discusses various experiences and opi...
8,2014_09_negitive,Several reports highlight serious adverse reac...
9,2014_10_negitive,The sentences discuss the controversy surround...


In [168]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdGlE8I84Wk8pLqY7ImauiKVbyXv at 0x26c2d3cadb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various aspects of vaccination, including the importance of keeping vaccinations up to date for children, the risks associated with certain vaccines for children with allergies (especially to milk), the low uptake of the HPV vaccine among girls, and reactions to the measles vaccine leading to its suspension in Amazonas. It also mentions criticism from the WHO regarding the lack of investment in vaccines for diseases like Ebola.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864155,
  "id": "chatcmpl-AUdGlE8I84Wk8pLqY7ImauiKVbyXv",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 78,
    "completion_to

## December

In [169]:
# Get monthly data
df_2014_12_neg = pd.read_csv('../data/local/df_neg_2014-12.csv').reset_index()
df_2014_12_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   466 non-null    int64  
 1   LocationCode            466 non-null    object 
 2   LocationName            466 non-null    object 
 3   StatesCode              466 non-null    object 
 4   ...4                    466 non-null    int64  
 5   id                      466 non-null    float64
 6   text                    466 non-null    object 
 7   label                   466 non-null    object 
 8   in_reply_to_status_id   56 non-null     float64
 9   in_reply_to_user_id     58 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       72 non-null     float64
 13  retweeted_status_id     72 non-null     float64
 14  created_at              466 non-null    ob

### Representative sample based on week and location

In [170]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2014_12_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2014_12_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2014-w48,Bahia,3,0.006438
1,2014-w48,Ceara,2,0.004292
2,2014-w48,Goias,2,0.004292
3,2014-w48,Mato Grosso,1,0.002146
4,2014-w48,Mato Grosso Do Sul,3,0.006438
...,...,...,...,...
83,2014-w52,Sao Paulo,17,0.036481
84,2014-w52,Sergipe,1,0.002146
85,2014-w53,Bahia,1,0.002146
86,2014-w53,Minas Gerais,1,0.002146


In [171]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2014-w48,Bahia,3,0.006438,15
1,2014-w48,Ceara,2,0.004292,10
2,2014-w48,Goias,2,0.004292,10
3,2014-w48,Mato Grosso,1,0.002146,5
4,2014-w48,Mato Grosso Do Sul,3,0.006438,15
...,...,...,...,...,...
83,2014-w52,Sao Paulo,17,0.036481,89
84,2014-w52,Sergipe,1,0.002146,5
85,2014-w53,Bahia,1,0.002146,5
86,2014-w53,Minas Gerais,1,0.002146,5


In [172]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2014_12_neg_sample_size = pd.merge(df_2014_12_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [173]:
# Step 6: Perform the stratified sampling
#df_2014_12_neg_sample = df_2014_12_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2014_12_neg_sample = df_2014_12_neg
# Step 7: Check the result
print(f"Sampled {len(df_2014_12_neg_sample)} rows.")


Sampled 466 rows.


In [174]:
text_gpt_2014_12_neg = '\n'.join(df_2014_12_neg_sample['text'].astype(str))

### Prompts and loops

In [175]:
prompt_12 = prompt_raw.replace('TWEETS', text_gpt_2014_12_neg)

In [176]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_12}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [177]:
response.choices[0].message.content

"The text discusses various opinions and reactions to vaccinations, highlighting concerns about pain, side effects, and the effectiveness of vaccines. It also addresses issues of public health, vaccination campaigns' failures, and societal attitudes toward vaccines, including resistance from some groups. The underlying themes include fear of needles, personal discomfort, and the broader implications of vaccination in society."

In [178]:
#summary = []
#dataset = []

In [179]:
dataset.append("2014_12_negative")

In [180]:
summary.append(response.choices[0].message.content)

In [181]:
df_2014_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2014_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  12 non-null     object
 1   summary  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [186]:
df_2014_neg.tail(14)

,dataset,summary
0,2014_01_negative,The sentences express various opinions and exp...
1,2014_02_negative,The text discusses various opinions and experi...
2,2014_03_negative,The text discusses the controversy surrounding...
3,2014_04_negative,The sentences discuss various sentiments about...
4,2014_05_negative,The text discusses various opinions and experi...
5,2014_06_negative,The text discusses various public reactions an...
6,2014_07_negative,The text discusses various experiences and opi...
7,2014_08_negative,The text discusses various experiences and opi...
8,2014_09_negative,Several reports highlight serious adverse reac...
9,2014_10_negative,The sentences discuss the controversy surround...


In [183]:
response

<OpenAIObject chat.completion id=chatcmpl-AUdGod4TbTLsEHQrAfuvAhAINUhla at 0x26c2d3d10e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and reactions to vaccinations, highlighting concerns about pain, side effects, and the effectiveness of vaccines. It also addresses issues of public health, vaccination campaigns' failures, and societal attitudes toward vaccines, including resistance from some groups. The underlying themes include fear of needles, personal discomfort, and the broader implications of vaccination in society.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731864158,
  "id": "chatcmpl-AUdGod4TbTLsEHQrAfuvAhAINUhla",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 69,
    "completion_tokens_details": {
     

## All 2014 negative

In [185]:
#df_2014_neg['dataset'] = df_2014_neg['dataset'].str.replace('negitive','negative')

In [187]:
df_2014_neg.to_csv('../data/summary_2014_negative_tweets.csv')